In [2]:
# Welcome to your new notebook
# Type here in the cell editor to add code!
df = spark.sql("SELECT * FROM lakehouse1.tbl_latest_news LIMIT 1000")
display(df)

StatementMeta(, d4d6ed03-bcda-4a37-a068-146c463d1d15, 4, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, d9790537-d00f-49f2-8a41-3cea3e0bb50e)

In [3]:
display(df.select("snippet"))

StatementMeta(, d4d6ed03-bcda-4a37-a068-146c463d1d15, 5, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 685b6712-e3cf-4636-84e6-7a75ab9bd8ca)

In [4]:
import synapse.ml.core
from synapse.ml.services import AnalyzeText

StatementMeta(, d4d6ed03-bcda-4a37-a068-146c463d1d15, 6, Finished, Available, Finished)

In [5]:
model = (AnalyzeText()
        .setTextCol("snippet")
        .setKind("SentimentAnalysis")
        .setOutputCol("response")
        .setErrorCol("Error"))

StatementMeta(, d4d6ed03-bcda-4a37-a068-146c463d1d15, 7, Finished, Available, Finished)

In [6]:
result = model.transform(df)

StatementMeta(, d4d6ed03-bcda-4a37-a068-146c463d1d15, 8, Finished, Available, Finished)

In [7]:
display(result)

StatementMeta(, d4d6ed03-bcda-4a37-a068-146c463d1d15, 9, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 037cad31-25bb-4c4c-b12b-f182fe3009ae)

In [8]:
display(result.select("snippet","response"))

StatementMeta(, d4d6ed03-bcda-4a37-a068-146c463d1d15, 10, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 6b17f99e-34b8-49a7-89b1-4e83d13aeed0)

In [9]:
from pyspark.sql.functions import col
sentiment_df = result.withColumn("Sentiment",col("response.documents.sentiment"))

StatementMeta(, d4d6ed03-bcda-4a37-a068-146c463d1d15, 11, Finished, Available, Finished)

In [10]:
display(sentiment_df)

StatementMeta(, d4d6ed03-bcda-4a37-a068-146c463d1d15, 12, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 320cc8a6-b51b-41f3-8114-e00a474dbe16)

In [11]:
df_final = sentiment_df.drop("error","response")

StatementMeta(, d4d6ed03-bcda-4a37-a068-146c463d1d15, 13, Finished, Available, Finished)

In [12]:
display(df_final)

StatementMeta(, d4d6ed03-bcda-4a37-a068-146c463d1d15, 14, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 27ca75b5-0f22-4294-a063-6ce13e15c10c)

In [13]:
display(df_final.select("snippet","Sentiment"))

StatementMeta(, d4d6ed03-bcda-4a37-a068-146c463d1d15, 15, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 475fc4b3-b0b5-430d-bc1f-1988ec31b556)

In [15]:
from pyspark.sql.utils import AnalysisException

table_name = "lakehouse1.tbl_sentiment_analysis"

spark.sql("CREATE DATABASE IF NOT EXISTS lakehouse1")

try:
    # Try creating the table fresh
    df_final.write.format("delta").saveAsTable(table_name)
    print(f"Table {table_name} created successfully.")

except AnalysisException:
    print(f"Table {table_name} already exists. Dropping and recreating...")

    # Drop the existing table
    spark.sql(f"DROP TABLE IF EXISTS {table_name}")

    # Recreate with df_final
    df_final.write.format("delta").saveAsTable(table_name)
    print(f"Table {table_name} recreated successfully.")

StatementMeta(, d4d6ed03-bcda-4a37-a068-146c463d1d15, 17, Finished, Available, Finished)

Table lakehouse1.tbl_sentiment_analysis created successfully.


In [16]:
%%sql
SELECT * FROM tbl_sentiment_analysis

StatementMeta(, d4d6ed03-bcda-4a37-a068-146c463d1d15, 18, Finished, Available, Finished)

<Spark SQL result set with 34 rows and 14 fields>